In [134]:
import requests
import json
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [135]:
curl = input("请输入curl命令：")

In [136]:
token_match = re.search(r'token:\s*([a-zA-Z0-9]+)', curl)
ic_cookie_match = re.search(r'ic-cookie=([a-zA-Z0-9-]+)', curl)
token = token_match.group(1)
ic_cookie = ic_cookie_match.group(1)

In [137]:
datetime_now = datetime.now().strftime('%Y%m%d')

In [138]:
url = 'http://libzwyy.jlu.edu.cn/ic-web/seatMenu'

headers = {
    'token': token,
}

cookies = {
    'ic-cookie': ic_cookie,
}

response = requests.get(url, headers=headers, cookies=cookies)
data_dict = json.loads(response.text)
learning_space_ids = []

def extract_ids(space):
    if "children" in space:
        for child in space["children"]:
            learning_space_ids.append(child["id"])

for floor in data_dict["data"]:
    extract_ids(floor)

In [139]:
import ipywidgets as widgets
from IPython.display import display

def on_change(selected_option):
    global option_value
    option_value = selected_option
    print(f"您选择的是: {selected_option}")

options = learning_space_ids

dropdown = widgets.interact(on_change, selected_option=options)

interactive(children=(Dropdown(description='selected_option', options=(100652799, 100652801, 100652803, 100652…

In [140]:
option_value

100652799

In [141]:
url = 'http://libzwyy.jlu.edu.cn/ic-web/reserve?roomIds='+str(option_value)+'&resvDates='+datetime_now+'&sysKind=8'

response = requests.get(url, headers=headers, cookies=cookies)

if response.status_code == 200:
    data = json.loads(response.text)
    devices_data = data["data"]
else:
    print(f"Error: Request failed with status code {response.status_code}")

In [142]:
data

{'code': 0,
 'message': '查询成功',
 'data': [{'devId': 100652812,
   'devSn': 100652812,
   'devName': '2B001',
   'minUser': 1,
   'maxUser': 1,
   'devStatus': 0,
   'devProp': 2,
   'coordinate': '50.419162,81.276596',
   'icon': None,
   'kindId': 100652789,
   'kindName': '座位',
   'roomId': 100652799,
   'roomSn': '1',
   'openRulesn': 1404,
   'openState': 1,
   'roomName': '开放式学习空间B',
   'roomProp': 0,
   'labId': 100652791,
   'labName': '二层',
   'labProp': 1,
   'openStart': '06:30',
   'openEnd': '22:00',
   'resvRule': {'uuid': '2f04ff0bcbf74470a7171b4290cd4b2f',
    'ruleId': 1,
    'ruleName': '座位预约规则',
    'ident': 0,
    'deptId': 0,
    'groupId': 0,
    'classKind': 8,
    'priority': 1,
    'limit': 260,
    'earlyInTime': 20,
    'earliestResvTime': 1440,
    'latestResvTime': 0,
    'minResvTime': 60,
    'maxResvTime': 930,
    'resvEndNoticeTime': 10,
    'seriesTimeLimit': 1440,
    'cancelTime': 30,
    'resvEndNewTime': 0,
    'resvBeforeNoticeTime': 10,
    'resv

In [143]:
# 提取 devName、startTime 和 endTime
table_data = []

for device in devices_data:
    dev_name = device["devName"]
    resv_info = device["resvInfo"]
    if len(resv_info) == 0:
        table_data.append([dev_name, 0, 0])
    for resv in resv_info:
        start_time = datetime.fromtimestamp(resv["startTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.fromtimestamp(resv["endTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        table_data.append([dev_name, start_time, end_time])
        print(start_time)

In [144]:
# 使用 pandas 创建 DataFrame
columns = ["devName", "startTime", "endTime"]
df = pd.DataFrame(table_data, columns=columns)

In [145]:
df["startTime"] = pd.to_datetime(df["startTime"])
df["endTime"] = pd.to_datetime(df["endTime"])
df['duration'] = (df['endTime'] - df['startTime']).dt.seconds / 3600

In [146]:
df.describe()

,duration
count,290.000000
mean,7.825000
std,4.754520
min,0.000000
25%,3.500000
50%,8.000000
75%,12.500000
max,15.166667


In [147]:
# 根据座位名称分组
groups = df.groupby("devName")

dev_names = df['devName'].unique()
name_to_index = {name: i for i, name in enumerate(dev_names)}

# 创建散点图
fig = go.Figure()

for name, group in groups:
    y_pos = name_to_index[name]
    for start_time, end_time in zip(group['startTime'], group['endTime']):
        fig.add_trace(
            go.Scatter(
                x=[start_time, end_time],
                y=[y_pos, y_pos],
                mode='lines',
                line=dict(color='orange', width=10),
                showlegend=False
            )
        )

# 设置x轴和y轴
fig.update_xaxes(title_text='时间', tickformat='%H:%M')
fig.update_yaxes(title_text='座位', tickvals=list(range(len(dev_names))), ticktext=dev_names)

# 设置布局
fig.update_layout(
    title='座位预约时间段分布',
    autosize=False,
    width=600,  # 调整图表宽度
    height=2400,  # 调整图表高度
    margin=dict(l=100, r=50, pad=4),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

# 显示图表
fig.show()